# Predicting Wine Cultivars

<br><br>

## Summary
---
In this project, we will predict what cultivar a wine was derived from based on its chemical properties.  

The data was sourced from the [UCI Machine Learning Repository](https://doi.org/10.24432/C5PC7J). It contains data about various wines from Italy derived from three different cultivars. Each row represents the chemical and physical properties of a different wine, such as its concentration of alcohol, magnesium level and hue. 

<br>

## Introduction
---
Wine is a beverage that has been enjoyed by humans for thousands of years (Feher et al. 2007). Consequently, humans have a long agricultural history with the grape plant which has led to the development of many different cultivars: grape plants selected and breed for their desirable characteristics (Harutyunyan and Malfeito-Ferreira 2022). Our dataset contains information about twelve chemical properties of 178 red wines made from three grape cultivars in Italy. 

The recorded chemical properties include: 
1. Alcohol content
2. Malic acid (gives the wine a fruity flavour)
3. Ash (left over inorganic matter from the wine-making process)
4. Alkalinity of ash (ability to resist acidification)
5. Magnesium, total phenols (contribute to bitter flavour of wine)
6. Flavanoids (antioxidants that contribute to bitter flavour and aroma of wine)
7. Nonflavanoid phenols (weakly acidic)
8. Proanthocyanins (bitter smell)
9. Color intensity
10. Hue
11. The ratio of OD280 to OD315 of diluted wines (protein concentration)
12. Proline (main amino acid in wine, important aspect of the flavour) (Bai et al. 2019).

Using this dataset, our predictive question is: **"What is the cultivar of an unknown wine based on the chemical properties?"** 

Identifying the chemical properties that distinguish cultivars enables farmers to make informed decisions about grape cultivation, aligning grape varieties with desired wine characteristics. By selecting cultivars known for specific flavor profiles or chemical compositions, farmers can tailor vineyard practices to meet market demands effectively. Moreover, this knowledge empowers brewers to experiment with wine compositions, fostering innovation and the creation of novel flavors. Armed with a deep understanding of wine chemistry, brewers can also strategically market their products, ensuring effective communication of the unique qualities and appeal of each wine to consumers.

<br>

## Code and Analysis
---

In [2]:
# imports and libraries 
library(GGally) # for ggpairs
library(tidyverse) #importing tidyverse
library(tidymodels) # importing tidymodels
library(repr)
library(gridExtra)
library(kknn)
library(dplyr) # for data wrangling
library(knitr) # to create tables
library(themis) # to balance our cultivar classes out

ERROR: Error in library(tidymodels): there is no package called ‘tidymodels’


In [ ]:
# Reading in the data from the web
set.seed(5)

raw_data <- read.csv("https://raw.githubusercontent.com/Kaylan-W/dsci_310_project/main/data/wine.data", header= FALSE)
           
# Name the columns based on the dataset description 
col_names <- c("cultivar","alcohol","malicacid", "ash", "alcalinity_of_ash", "magnesium", "total_phenols", 
               "flavanoids", "nonflavanoid_phenols", "proanthocyanins", "color_intensity", "hue", "0D280_0D315_ratio", "proline")
colnames(raw_data) <- col_names


<br><br>

## Exploratory Data Analysis

In [ ]:
# EDA: use a plot to see the relationships between variables
options(repr.plot.width = 12, repr.plot.height = 30) # format the any visualizations to be easily viewable
pairplots <- raw_data %>%
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )
pairplots

Figure 1: A correlation matrix for the variables in our dataset.

<br>

In [ ]:
# Check the number of observations per cultivar
sample_size_cultivar <- raw_data %>%
    group_by(cultivar) %>%
    summarize(sample_size = n())

sample_size_cultivar # class 2 has more observations than the other classes -> need to balance the classes out

Table 1: The number of observations for each level of `cultivar` before balancing.

<br>

In [ ]:
# balance the cultivar class 

# convert cultivar into a factor
raw_data <- raw_data |>
  mutate(cultivar = as.factor(cultivar))

# recipe for balancing
cultivar_balance_recipe <- recipe(cultivar ~ ., data = raw_data) |>
  step_upsample(cultivar, over_ratio = 1, skip = FALSE) |>
  prep()

# execute the balancing
data <- bake(cultivar_balance_recipe, raw_data)

# check the data is balanced
balanced_data <- data |>
  group_by(cultivar) |>
  summarize(n = n())

balanced_data # data has been upsampled so all groups have equal sample size


Table 2: The number of observations for each level of `cultivar` after balancing.

<br>

In [ ]:
# calculate the mean for each cultivar group
cultivar_mean_table <- raw_data |>
    group_by(cultivar) |>
    summarize(across(alcohol:proline, mean, na.rm = TRUE))

cultivar_mean_table <- cultivar_mean_table %>% 
                      mutate(stat = 'mean') %>%
                      select(stat, everything())

 
# calculate the standard deviation for each cultivar group
cultivar_sd_table <- raw_data |>
    group_by(cultivar) |>
    summarize(across(alcohol:proline, sd, na.rm = TRUE))

cultivar_sd_table <- cultivar_sd_table %>% 
                      mutate(stat = 'sd') %>%
                      select(stat, everything())

# calculate the maximum values for each cultivar group
cultivar_max_table <- data %>%
  group_by(cultivar) %>%
  summarize(across(alcohol:proline, ~max(.x, na.rm = TRUE))) %>%  # Corrected line
  mutate(stat = 'max') %>%
  select(stat, everything())

# # calculate the minimum values for each cultivar group
cultivar_min_table <- raw_data |>
    group_by(cultivar) |>
    summarize(across(alcohol:proline, min, na.rm = TRUE))

cultivar_min_table <- cultivar_min_table %>% 
                      mutate(stat = 'min') %>%
                      select(stat, everything())


appended_stats_df <- rbind(cultivar_mean_table,cultivar_sd_table, cultivar_max_table, cultivar_min_table)
appended_stats_df

Table 3: Summary statistics for each variable per cultivar.

<br>

In [ ]:
eda_plot_data <- raw_data |>
    relocate(flavanoids, 1)

eda_plot_data <- eda_plot_data |>
        pivot_longer(
        cols= alcohol:proline,
        names_to="factor",
        values_to="values")

eda_plot1 <- eda_plot_data|>
    ggplot(aes(x=flavanoids,y=values,color=cultivar))+
    geom_point(alpha=0.35)+
    facet_grid(factor~.,scales="free")+
    labs(x="Flavanoid",y="Values",color="Cultivar")
eda_plot1


Figure 2: Colourizing `cultivar` for each variable in the dataset to visualize patterns in the data.

<br>

## K-nearest neighbours regression

In [ ]:
data_split <- initial_split(data, prop = 0.75, strata = cultivar)
data_train <- training(data_split)
data_test <- testing(data_split)

In [ ]:
data_recipe <- recipe(cultivar ~ flavanoids + total_phenols, data = data_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

data_spec <- nearest_neighbor(weight_func = "rectangular",
                               neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

data_vfold <- vfold_cv(data_train, v = 5, strata = cultivar)

k_vals <- tibble(neighbors = seq(from = 1, to = 70, by = 1))

knn_results <- workflow() |>
    add_recipe(data_recipe) |>
    add_model(data_spec) |>
    tune_grid(resamples = data_vfold, grid = k_vals) |>
    collect_metrics()

accuracies = knn_results |>
    filter(.metric == "accuracy")

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "K (neighbors)", y = "Accuracy Estimate (%)", title = "Accuracy vs Number of Neighbors (k)") +
    theme(text = element_text(size = 25))

options(repr.plot.width = 15, repr.plot.height = 15) # format the any visualizations to be easily viewable
cross_val_plot

Figure 3: A comparison of $k$ and accuracy.

<br>

In [ ]:
knn_spec_final <- nearest_neighbor(weight_func = "rectangular", neighbors = 8) |>
    set_engine("kknn") |>
    set_mode("classification")


final_fit<-workflow()|>
        add_recipe(data_recipe)|>
        add_model(knn_spec_final)|>
        fit(data_train)
final_fit

Table 4: The hyperparameters of the optimized k-nn model.

<br>

In [ ]:
data_test_predictions<-predict(final_fit,data_test)|>
    bind_cols(data_test)|>
    arrange(cultivar)|>
    mutate(Class=as_factor(cultivar))

data_prediction_accuracy <- data_test_predictions |>
        metrics(truth = Class, estimate = .pred_class)|>
        filter(.metric=="accuracy")

data_prediction_accuracy

Table 5: A summary of the model's accuracy.

<br>

In [ ]:
data_prediction_truth_mat<-data_test_predictions |>
        conf_mat(truth = Class, estimate = .pred_class)

data_prediction_truth_mat

Table 6: A confusion matrix for the model's predictions.

<br>

<br>

## Methods & Results
---
This project utilized a K-nearest neighbours algorithm to predict what cultivar a wine was derived from based on its various chemical properties. First, we read in data from the UCI Machine Learning Repository. It contains data about various wines from Italy derived from three different cultivars. Each row represents the chemical and physical properties of a different wine, such as its concentration of alcohol, magnesium level and hue.

We then tidied the data and balanced the classes of the classification variable we are interested in. This is because the data set is not extensively large, so ensuring each class has an equal number of observations prevents our model from being biased towards a specific dominant class. Next we calculated some summary statistics to facilitate exploratory data analysis, with the goal of finding key input variables for our model.

The data was split into 75% for the training set and 25% for the test set. To fine tune our model, we used 5 fold cross validation, grid search, and graphical methods to choose the optimal value of $K$. The result was $K$ = 8 being used in the k-nn model.


<br>

## Discussion
--- 

Our multiclass k-nn model performed relatively well on the test data, achieving an accuracy estimate of approximately $80\%$. The confusion matrix reveals insights into the model's performance across the three cultivar classes. Notably, while the model demonstrated strong precision and recall for predicting cultivar 3, it encountered challenges in accurately classifying cultivar 2. This aligns with our initial hypothesis that certain chemical properties may serve as distinguishing factors for wine cultivars.

However, despite the model's overall success, its limitations in predicting cultivar 2 suggest avenues for improvement. Future iterations of the model could benefit from refining input variables to better capture the nuances of each cultivar's chemical composition. Moreover, our findings underscore the importance of further investigation into the unique characteristics of cultivar 3, which consistently stood out in our predictions.

By elucidating the chemical properties that differentiate wine cultivars, our study contributes to the broader goal of simplifying wine classification for consumers. Ultimately, this research not only enhances our understanding of wine chemistry but also has practical implications for wine enthusiasts and industry professionals alike. 

<br>

## References
---
Aeberhard,Stefan and Forina,M.. (1991). Wine. UCI Machine Learning Repository. https://doi.org/10.24432/C5PC7J.

Bai, X., Wang, L., & Li, H. (2019). Identification of red wine categories based on physicochemical properties. International Conference on Educational Technology, Management, and Humanities Science, 1443-1448.
https://doi.org/ 10.25236/etmhs.2019.30



Fehér, J., Lengyel, G., & Lugasi, A. (2007). The cultural history of wine—Theoretical background to wine therapy. Central European Journal of Medicine, 2(4), 379–391. https://doi.org/10.2478/s11536-007-0048



Harutyunyan, M., & Malfeito-Ferreira, M. (2022). The Rise of Wine among Ancient Civilizations across the Mediterranean Basin. Heritage, 5(2), Article 2. https://doi.org/10.3390/heritage50203


